<a href="https://colab.research.google.com/github/jbarker6706/Portfolio9/blob/master/Assignment9ROSTB3Wander.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Here is one of two files used to get turtlebot3 wandering this contains the logic turtlebot3_logic.py

In [0]:
#!/usr/bin/python

import math
import rospy
from geometry_msgs.msg import Twist
from sensor_msgs.msg import LaserScan


class RosTurtlebotLogic(object):

    def __init__(self, forwardSpeed, rotationSpeed, minDistanceFromObstacle, minimumAngle, maximumAngle):
        self.forwardSpeed = forwardSpeed
        self.rotationSpeed = rotationSpeed
        self.minDistanceFromObstacle = minDistanceFromObstacle

        # Keeps the current minimum distance from obstacle from laser.
        self.minimumRangeAhead = 5
        # In which direction to rotate, left or right.
        self.rotationDirection = .1
#        self.minimumIndexLaser = 360 * (90 + minimumAngle) / 90
#        self.maximumIndexLaser = 360 * (90 + maximumAngle) / 90 - 1
        self.minimumIndexLaser = 0
        self.maximumIndexLaser = 358
        self.keepMoving = True

       # self.commandPub = rospy.Publisher("/cmd_vel_mux/input/teleop", Twist, queue_size=10)
        self.commandPub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
        rospy.Subscriber("scan", LaserScan, self.scanCallback, queue_size=10)

    def startMoving(self):
        rate = rospy.Rate(10)
        count = 0

        while(not rospy.is_shutdown()):
            if(self.keepMoving):
                if (self.minimumRangeAhead < self.minDistanceFromObstacle):
                    #print("I am GROOT <")
                    self.keepMoving = False
            else:
                if(self.minimumRangeAhead > self.minDistanceFromObstacle):
                    #print("I am GROOT >")
                    self.keepMoving = True

            twist = Twist()
            if(self.keepMoving):
                #print("I am GROOT x")
                twist.linear.x = self.forwardSpeed
                twist.angular.z = 0
            else:
                #print("I am GROOT z")
                if(count%2==0):
                    twist.angular.z = self.rotationDirection
                    twist.linear.x = 0
                else:
                    twist.angular.z = 0
                    twist.linear.x = 0

                self.keepMoving = True
            print(twist.linear.x, twist.angular.z)

            self.commandPub.publish(twist)
            rate.sleep()

    def scanCallback(self, scan_msg):
        minimum = 100
        index = 0
        
        # Find the minimum distance to obstacle and we keep the minimum distance and the index.
        # We need the minimum distance in order to know if we can go forward or not.
        # We need the index to know in which direction to rotate.
        if(not math.isnan(scan_msg.ranges[self.minimumIndexLaser])):
            minimum = scan_msg.ranges[self.minimumIndexLaser]

        for i in range(self.minimumIndexLaser, self.maximumIndexLaser + 1):
            if(not math.isnan(scan_msg.ranges[i]) and scan_msg.ranges[i] < minimum):
                minimum = scan_msg.ranges[i]
                index = i

        self.minimumRangeAhead = minimum

        if(index <= 359):
            self.rotationDirection = self.rotationSpeed
        else:
            self.rotationDirection = -self.rotationSpeed

This is the file which is the ROS node

In [0]:
#!/usr/bin/python

import rospy
import sys
from turtlebot3_logic import RosTurtlebotLogic


def loadParams():
    forwardSpeed = 100
    rotationSpeed = 10
    minDistanceFromObstacle = 1.0
    minimumAngle = -30
    maximumAngle = 30

    if rospy.has_param('~forward_speed'):
        forwardSpeed = rospy.get_param('~forward_speed')
        print("fs")
    if rospy.has_param('~rotation_speed'):
        rotationSpeed = rospy.get_param('~rotation_speed')
        print("rs")
    if rospy.has_param('~minimum_distance_from_obstacle'):
        minDistanceFromObstacle = rospy.get_param('~minimum_distance_from_obstacle')
        print("mindis")
    if rospy.has_param('~minimum_angle'):
        minimumAngle = rospy.get_param('~minimum_angle')
        print("minang")
    if rospy.has_param('~maximum_angle'):
        maximumAngle = rospy.get_param('~maximum_angle')
        print("maxang")

    return forwardSpeed, rotationSpeed, minDistanceFromObstacle, minimumAngle, maximumAngle


if __name__ == "__main__":
    rospy.loginfo("Started program.")

    rospy.init_node("stopper_node", argv=sys.argv)
    forwardSpeed, rotationSpeed, minDistanceFromObstacle, minimumAngle, maximumAngle = loadParams()
    rospy.loginfo("Finished import parameters.")
    
    #print(forwardSpeed, rotationSpeed, minDistanceFromObstacle, minimumAngle, maximumAngle)

    my_stopper = RosTurtlebotLogic(forwardSpeed, rotationSpeed, minDistanceFromObstacle, minimumAngle, maximumAngle)
    my_stopper.startMoving()

I had two helper nodes because I was struggling to get the bot to wander without getting stuck. So I had a forward backward node trajectory.py.
I also had a rotate node rturtlebot.py. Both useful in unsticking my bot

In [0]:
#!/usr/bin/python

import rospy
from geometry_msgs.msg import Twist

def talker():
    rospy.init_node('vel_publisher')
    pub = rospy.Publisher('cmd_vel', Twist, queue_size=10)
    move = Twist();
    rate = rospy.Rate(1)

    while not rospy.is_shutdown():
       move.linear.x = 10
       move.angular.z = 0
       pub.publish(move)
       rate.sleep()

if __name__ == '__main__':
    try:
        talker()
    except rospy.ROSInterruptException:
        pass


In [0]:
#!/usr/bin/python
import rospy
from nav_msgs.msg import Odometry
from tf.transformations import euler_from_quaternion, quaternion_from_euler
from geometry_msgs.msg import Twist
import math

roll = pitch = yaw = 0.0
target = 90
kp=0.5

def get_rotation (msg):
    global roll, pitch, yaw
    orientation_q = msg.pose.pose.orientation
    orientation_list = [orientation_q.x, orientation_q.y,         orientation_q.z, orientation_q.w]
    (roll, pitch, yaw) = euler_from_quaternion (orientation_list)
    print yaw

rospy.init_node('rotate_robot')

sub = rospy.Subscriber ('/odom', Odometry, get_rotation)
pub = rospy.Publisher('cmd_vel', Twist, queue_size=1)
r = rospy.Rate(10)
command =Twist()

while not rospy.is_shutdown():
    target_rad = target*math.pi/180
    command.angular.z = kp * (target_rad-yaw)
    pub.publish(command)
    print("taeget={} current:{}", target,yaw)
    r.sleep()